In [1]:
from dask import dataframe as dd
from dask.multiprocessing import get
from multiprocessing import cpu_count
nCores = cpu_count()
from preprocess import *
import gc

import pandas as pd
import numpy as np
import os
import glob
import time

Using TensorFlow backend.


In [2]:
# Path definitions
base_dir = 'input'
ALL_TRAIN_PATHS = glob.glob(os.path.join(base_dir, 'train_simplified/*.csv'))

In [3]:
all_dfs = []

for p in ALL_TRAIN_PATHS[:3]:
    all_dfs.append(pd.read_csv(p)[['word', 'drawing']])
    
full_df = pd.concat(all_dfs)
print("Full shape", full_df.shape)
full_df.info(memory_usage='deep')

('Full shape', (588154, 2))
<class 'pandas.core.frame.DataFrame'>
Int64Index: 588154 entries, 0 to 178285
Data columns (total 2 columns):
word       588154 non-null object
drawing    588154 non-null object
dtypes: object(2)
memory usage: 306.2 MB


In [4]:
full_df = full_df.reset_index(drop=True)

In [ ]:
dask.config.set(scheduler='processes')

ddf = dd.from_pandas(full_df,npartitions=nCores)

ddf['drawing'] = ddf['drawing']. \
                        map_partitions(
                              lambda d : d.apply(stroke_vector), meta=object)

ddf.to_csv('input/train_processed/test-*.csv') 

### https://towardsdatascience.com/how-i-learned-to-love-parallelized-applies-with-python-pandas-dask-and-numba-f06b0b367138

In [5]:
gc.collect()

start = time.time()

# ALWAYS use processes scheduler
full_df['drawing'] = dd.from_pandas(full_df['drawing'],npartitions=nCores).\
   map_partitions(
      lambda d : d.apply(stroke_vector), meta=object).\
   compute(scheduler='processes')

print(time.time() - start)
full_df.info(memory_usage='deep')

222.567875147
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558365 entries, 0 to 1558364
Data columns (total 2 columns):
word       1558365 non-null object
drawing    1558365 non-null object
dtypes: object(2)
memory usage: 254.1 MB


In [7]:
test = full_df['drawing'][0]

test.shape

(196, 3)